#### Spark经历了从hash shuffle到sort shuffle的演变, 下面详细介绍这些不同shuffle的特点优势

### 一. 普通hash shuffle
####  shuffle write
* 通过算子划分stage后, 下一个stage的task有多少个, 当前stage的每个task就要产生多少个文件, 以供下个stage的executor拉取数据.  
    从partition个数和exector线程数的角度看: 若下一个stage有n个分区,每个分区启动k个线程执行task, 则上一个stage的每个task都要产生nk个文件
* 比如下一个stage总共有100个task，那么当前stage的每个task都要创建100份磁盘文件。如果当前stage有50个task，总共有10个Executor，每个Executor执行5个Task，那么每个Executor上总共就要创建500个磁盘文件，所有Executor上会创建5000个磁盘文件。由此可见，普通的hash shuffle write操作所产生的磁盘文件的数量是极其惊人的。

#### shuffle read
* shuffle read通常是一个stage刚开始要做的事情. 此时, 每个task都需要拉取上一个stage输出结果中相同的key的记录, 然后根据key进行聚合,连接等操作. 因此每个task都要到上一个stage的所有task所在节点上, 拉取属于自己的文件
* shuffle read是边拉取边聚合. 因此, 需要read缓冲区, 每次只拉取与缓冲区相同大小的数据, 然后再内存中聚合, 而后再次拉取数据, 直到所有数据拉取完毕, 得到最终结果

### 二. 开启consolidate的hash shuffle
#### shuffle write
* 开启consolidate后, 不再是每个task产生为下个stage的task产生各自的文件, 而是同一批次的task为下个stage的task产生各自的文件,让不同批次的task服用同一批磁盘文件
* 假设第二个stage有100个task，第一个stage有50个task，总共还是有10个Executor，每个Executor执行5个task。那么原本使用未经优化的HashShuffleManager时，每个Executor会产生500个磁盘文件，所有Executor会产生5000个磁盘文件的。但是此时经过优化之后，每个Executor创建的磁盘文件的数量的计算公式为：CPU core的数量 * 下一个stage的task数量。也就是说，假设有4个core, 每个Executor此时只会创建400个磁盘文件，所有Executor只会创建4000个磁盘文件。

### 三. sort shuffle
#### shuffle write
* 该模式下, 当前stage的每个task会持有一个map缓冲区, 当缓冲区满后, 将数据排序然后溢写到磁盘上. 最后把这些溢写文件merge成一个大文件, 为了便于下个stage的task拉取数据, 还会配套一个索引文件, 记录下游每个task拉取数据的start offset和end offset
* sort shuffle模式下, 每个map task输出一个文件和一个索引文件
* 溢写机制可以使到达缓冲区大小, 也可以是记录数达到一定规模, 默认10000条
* 根据不同的shuffle算子，可能选用不同的数据结构。如果是reduceByKey这种聚合类的shuffle算子，那么会选用Map数据结构，一边通过Map进行聚合，一边写入内存；如果是join这种普通的shuffle算子，那么会选用Array数据结构
* 比如第一个stage有50个task，总共有10个Executor，每个Executor执行5个task，而第二个stage有100个task。由于每个task最终只有一个磁盘文件，因此此时每个Executor上只有5个磁盘文件，所有Executor只有50个磁盘文件。

### 四. bypass Sort Shuffle
#### shuffle write
* bypass和普通hash shuffle一样, 每个task会产生于下个stage task数量相同的临时文件, 也是使用hash值将记录写到不同的临时文件中.  
 不同的是, bypass最后会将这些临时文件merge成一个文件, 保证每个map task最终产生一个文件, 减小shuffle read的时间
* bypass根据key的hash值溢写的方式, 也是先写到内存缓冲区, 再写到对应文件
* 该机制与普通SortShuffleManager运行机制的不同在于：
    * 第一，磁盘写机制不同；
    * 第二，不会进行排序。  
也就是说，启用该机制的最大好处在于，shuffle write过程中，不需要进行数据的排序操作，也就节省掉了这部分的性能开销。

shuffle相关的参数调优
--------
`spark.shuffle.file.buffer`
* 默认值：32k
* 参数说明：  
该参数用于设置shuffle write task的BufferedOutputStream的buffer缓冲大小。将数据写到磁盘文件之前，会先写入buffer缓冲中，待缓冲写满之后，才会溢写到磁盘。
* 调优建议：   
如果作业可用的内存资源较为充足的话，可以适当增加这个参数的大小（比如64k），从而减少shuffle write过程中溢写磁盘文件的次数，也就可以减少磁盘IO次数，进而提升性能。在实践中发现，合理调节该参数，性能会有1%~5%的提升。

`spark.reducer.maxSizeInFlight`
* 默认值：48m
* 参数说明：  
该参数用于设置shuffle read task的buffer缓冲大小，而这个buffer缓冲决定了每次能够拉取多少数据。
* 调优建议：  
如果作业可用的内存资源较为充足的话，可以适当增加这个参数的大小（比如96m），从而减少拉取数据的次数，也就可以减少网络传输的次数，进而提升性能。在实践中发现，合理调节该参数，性能会有1%~5%的提升。

`spark.shuffle.io.maxRetries`
* 默认值：3
* 参数说明：  
shuffle read task从shuffle write task所在节点拉取属于自己的数据时，如果因为网络异常导致拉取失败，是会自动进行重试的。该参数就代表了可以重试的最大次数。如果在指定次数之内拉取还是没有成功，就可能会导致作业执行失败。
* 调优建议：  
对于那些包含了特别耗时的shuffle操作的作业，建议增加重试最大次数（比如60次），以避免由于JVM的full gc或者网络不稳定等因素导致的数据拉取失败。在实践中发现，对于针对超大数据量（数十亿~上百亿）的shuffle过程，调节该参数可以大幅度提升稳定性。

`spark.shuffle.io.retryWait`
* 默认值：5s
* 参数说明：具体解释同上，该参数代表了每次重试拉取数据的等待间隔，默认是5s。
* 调优建议：建议加大间隔时长（比如60s），以增加shuffle操作的稳定性。

`spark.shuffle.memoryFraction`
* 默认值：0.2
* 参数说明：该参数代表了Executor内存中，分配给shuffle read task进行聚合操作的内存比例，默认是20%。
* 调优建议：在资源参数调优中讲解过这个参数。如果内存充足，而且很少使用持久化操作，建议调高这个比例，给shuffle read的聚合操作更多内存，以避免由于内存不足导致聚合过程中频繁读写磁盘。在实践中发现，合理调节该参数可以将性能提升10%左右。

`spark.shuffle.manager`
* 默认值：sort
* 参数说明：该参数用于设置ShuffleManager的类型。Spark 1.5以后，有三个可选项：hash、sort和tungsten-sort。HashShuffleManager是Spark 1.2以前的默认选项，但是Spark 1.2以及之后的版本默认都是SortShuffleManager了。tungsten-sort与sort类似，但是使用了tungsten计划中的堆外内存管理机制，内存使用效率更高。
* 调优建议：由于SortShuffleManager默认会对数据进行排序，因此如果你的业务逻辑中需要该排序机制的话，则使用默认的SortShuffleManager就可以；而如果你的业务逻辑不需要对数据进行排序，那么建议参考后面的几个参数调优，通过bypass机制或优化的HashShuffleManager来避免排序操作，同时提供较好的磁盘读写性能。这里要注意的是，tungsten-sort要慎用，因为之前发现了一些相应的bug。

`spark.shuffle.sort.bypassMergeThreshold`
* 默认值：200
* 参数说明：当ShuffleManager为SortShuffleManager时，如果shuffle read task的数量小于这个阈值（默认是200），则shuffle write过程中不会进行排序操作，而是直接按照未经优化的HashShuffleManager的方式去写数据，但是最后会将每个task产生的所有临时磁盘文件都合并成一个文件，并会创建单独的索引文件。
* 调优建议：当你使用SortShuffleManager时，如果的确不需要排序操作，那么建议将这个参数调大一些，大于shuffle read task的数量。那么此时就会自动启用bypass机制，map-side就不会进行排序了，减少了排序的性能开销。但是这种方式下，依然会产生大量的磁盘文件，因此shuffle write性能有待提高。

`spark.shuffle.consolidateFiles`
* 默认值：false
* 参数说明：如果使用HashShuffleManager，该参数有效。如果设置为true，那么就会开启consolidate机制，会大幅度合并shuffle write的输出文件，对于shuffle read task数量特别多的情况下，这种方法可以极大地减少磁盘IO开销，提升性能。
* 调优建议：如果的确不需要SortShuffleManager的排序机制，那么除了使用bypass机制，还可以尝试将spark.shffle.manager参数手动指定为hash，使用HashShuffleManager，同时开启consolidate机制。在实践中尝试过，发现其性能比开启了bypass机制的SortShuffleManager要高出10%~30%。

